## Hierarchy Classificator Complete Study

With all the information gathered with `HierarchyClassificatorLevelStudy.ipynb` we've created the 3 layer model called `HerarchyModel.py` that takes 3 trained models and uses this herarchy structure to predict different attacks.

In [7]:
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install seaborn
#!pip install scikit-learn
#!pip install fastparquet
#!pip install tensorflow
#!pip install keras
#!pip uninstall xgboost
#!pip install xgboost

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder

from HerarchyModel.HerarchyModel import HerarchyModel

from fastparquet import ParquetFile

For this case, lvl3 model requires a smaller number of variables. We need to find this value to pass it to the model

In [9]:
dataWeb = pd.read_csv('Datasets/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
dataWeb.columns = dataWeb.columns.str.strip()  # Remove leading/trailing spaces
dataWeb.rename(columns={'Label': 'AttackType'}, inplace=True)

data = pd.read_parquet('Datasets/Complete/cic-collection.parquet')
data.rename(columns={'Label': 'AttackType'}, inplace=True)

In [13]:
dataDdos = pd.read_csv('Datasets/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
dataPortScan = pd.read_csv('Datasets/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')

dataDdos.columns = dataDdos.columns.str.strip()  # Remove leading/trailing spaces
dataDdos.rename(columns={'Label': 'AttackType'}, inplace=True)

dataPortScan.columns = dataPortScan.columns.str.strip()  # Remove leading/trailing spaces
dataPortScan.rename(columns={'Label': 'AttackType'}, inplace=True)

dataAll = pd.concat([dataWeb, dataDdos, dataPortScan], ignore_index=True)

dataAll = dataAll.dropna()
dataAll.columns = dataAll.columns.str.strip()  # Remove leading/trailing spaces
columns_dataAll = set(dataAll.columns)
columns_data = set(data.columns)
shared_columns = columns_data.intersection(columns_dataAll)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


In [11]:
X = dataAll.drop(columns=['AttackType'])
y = dataAll['AttackType']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
unique, counts = np.unique(y_encoded, return_counts=True)


In [12]:
herarchy = HerarchyModel()

herarchy.load_models('LayerModels/lvl1_rf_model.pkl','LayerModels/lvl2_nn_model.keras', 'LayerModels/lvl3_xgboost_model.pkl', lvl3_columns=shared_columns)

y_pred = herarchy.predict(X)

print(classification_report(y_encoded, y_pred, target_names=label_encoder.classes_))

cm = confusion_matrix(y_encoded, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)

disp.plot()

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ACK Flag Count
- Active Max
- Active Min
- Active Std
- Average Packet Size
- ...
Feature names seen at fit time, yet now missing:
-  ACK Flag Count
-  Active Max
-  Active Min
-  Active Std
-  Average Packet Size
- ...
